## Libraries

In [1]:
from time import time
import sys
import os
import glob
import csv
import math

from six.moves import xrange
import tensorflow as tf
import numpy as np
import Image


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

DATA_PATH = '../data/statefarmchallenge/'

## Convert to recoreds

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _byte_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def convert(images, labels, name):
    """Convert images and labesl to TFRecord"""
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("images size {} did not match labels size {}"
                         .format(images.shape[0], num_examples))
        
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    filename = os.path.join(DATA_PATH, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(labels[index]),
            'image_raw': _byte_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()
       

def load_image(infilename):
    """Load our image and return as numpy array"""
    img = Image.open(infilename)
    data = np.asarray(img)
    return data


def load(rows, data_path):
    """Load a batch of images & labels for converting"""
    images = []
    labels = []
    
    print('Loading batch')
    for row in rows:
        path = os.path.join(data_path, row[1], row[2])
        data = load_image(path)
        images.append(data)
        labels.append(int(row[1][1]))
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels



def gen_record(size, records, name, num_batches, data_path):
    """Generate a record file by batching data"""
    examples_per_batch = size // num_batches
    
    if examples_per_batch <= 1:
        raise ValueError('Batch size must be greater than 1')
        
    print("Generating " + name + " record")
    for i in xrange(num_batches):
        k = i * examples_per_batch
        record_b = records[k : k + examples_per_batch]

        images_b, labels_b = load(record_b, data_path)

        convert(images_b, labels_b, name)
                
    # What about left overs???
    records_processed = examples_per_batch * num_batches
    left_over = size - records_processed
    
    if left_over > 0:
        record_b = records[records_processed:]
        images_b, labels_b = load(record_b, data_path)
        convert(images_b, labels_b, name)
        
        

def read_csv(path):
    """Read CSV record and return as list"""
    print('Reading csv')
                
    with open(path, 'rb') as f:
        reader = csv.reader(f)
        records = list(reader)
    
    return records
        

    
def main(argv):
    records = read_csv(DATA_PATH + 'driver_imgs_list.csv')
    num_examples = len(records)
    
    # Generate validation TFRecords (20%)
    validation_size = num_examples // 5
    val_records = records[ : validation_size]
    gen_record(size=validation_size, 
               records=val_records, 
               name='validation', 
               num_batches=5, 
               data_path=DATA_PATH + 'imgs/train/')

    
    # Generate training TFRecords
    train_size = num_examples - validation_size
    train_records = records[validation_size : ]
    gen_record(size=train_size, 
               records=train_records, 
               name='train', 
               num_batches=10, 
               data_path=DATA_PATH + 'imgs/train')
                


if __name__ == '__main__':
    tf.app.run()

Reading csv
Generating validation record
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Loading batch
Writing ../data/statefarmchallenge/validation.tfrecords
Generating train record
Loading batch


## Hyper Parameters

In [32]:
num_classes = 10

# Convolution output depths 
depth_1 = 16
depth_2 = 64
depth_3 = 256

# shapes
channels = 3

# Local layer size
num_hidden = 2048

# Learning rate
learning_rate = .01
decay_rate = .9
decay_setps = 1000

# Regularization
beta = 1e-3

# batching and images size
batch_size = 256
epochs = 2
height = 480
width = 640
depth = 3
image_size = height * width * depth

## Input Pipeline 

The pipeline queues up file names decodes and preprocess images and then shuffles them.  

**NOTE** Try using parse_example for batching and skip post batching/shuffling

In [37]:
TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'validation.tfrecords'
    
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
            'height' : tf.FixedLenFeature([], tf.int64),
            'width' : tf.FixedLenFeature([], tf.int64),
            'depth' : tf.FixedLenFeature([], tf.int64),
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
      })

    # Convert from a scalar string to unit8 tensor
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.reshape(image, [height, width, depth])

    # OPTIONAL: Could reshape image and apply distortions or whitening here


    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(image, tf.float32)
    image = tf.sub(tf.mul(image, 3.921568e-3), 0.5)

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)

    return image, label
    
def inputs(train, batch_size, num_epochs):
    if not num_epochs: num_epochs = None
    filename = os.path.join(DATA_PATH, TRAIN_FILE if train else VALIDATION_FILE)

    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)

    # Even when reading in multiple threads, share the filename
    # queue.
    image, label = read_and_decode(filename_queue)
    
    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in 8 threads to avoid being a bottleneck.
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=8,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)

    return images, sparse_labels

## Helper Functions

In [38]:
def _dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

def _variabel_on_cpu(name, shape, initializer, regularizer=None):
    """Helper to create a Variable stored on CPU memory."""
    with tf.device('\cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer, regularizer=regularizer)
    
    return var

def local_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        weights = _variable_on_cpu("weights", [input_dim, output_dim],
            tf.truncated_normal_initializer(stddev=2.0 / float(input_dim)), 
            regularizer=tf.contrib.layers.l2_regularizer(beta))
        
        biases = _variable_on_cpu("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        
        preactivate = tf.add(tf.matmul(input_tensor, weights), biases)
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
            
    activations = act(preactivate, 'activation')
    tf.histogram_summary(layer_name + '/activations', activations)
    return activations

def conv_relu(input, kernel_shape, bias_shape, layer_name):
    """Reusable code for convoltional layers
    
    It odes the 2D convolution and relu
    """
    with tf.name_scope(layer_name):
        # Create variable named "weights".
        weights = _variabel_on_cpu("weights", kernel_shape,
            initializer=tf.truncated_normal_initializer(stddev=1e-4),
            regularizer=tf.contrib.layers.l2_regularizer(beta))

        # Create variable named "biases".
        biases = _variabel_on_cpu("biases", [bias_shape], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(tf.add(conv, biases))

## Model

*TODO:*
- ~~Add variable reuse~~
- ~~add images to summary~~
- ~~change learning rate to exp step~~
- Test tensorboard
- ~~clean up local layers~~
- ~~update local initialization~~
- consider 1x1 convolutions
- and insception ie. pool --> 1x1 conv --> 1x1, 3x3, 5x5 --> concat all outputs

In [35]:
def inference(images, keep_prob):
    """
    Args:
    images: Images placeholder, from inputs().
    Returns:
    softmax_linear: Output tensor with the computed logits.
    """

    # Conv1 /w pooling
    conv_1 = conv_relu(images, [5, 5, channels, depth_1], [depth_1], 'conv_1')     
    pool_1 = tf.nn.avg_pool(conv_1, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME')
    norm_1 = tf.nn.local_response_normalization(pool_1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
        
    # Conv2 /w pooling 
    conv_2 = conv_relu(norm_1, [3, 3, depth_1, depth_2], [depth_2], 'conv_2')     
    pool_2 = tf.nn.avg_pool(conv_2, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME')
    norm_2 = tf.nn.local_response_normalization(pool_2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    # Conv3 /w pooling
    conv_3 = conv_relu(norm_2, [3, 3, depth_2, depth_3], [depth_3], 'conv_3')     
    pool_3 = tf.nn.avg_pool(conv_3, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME')
    norm_3 = tf.nn.local_response_normalization(pool_3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    shape = norm_3.get_shape().as_list()
    dim = tf.mul(shape[1], tf.mul(shape[2], shape[3]))
    reshape = tf.reshape(norm_3, [shape[0], dim])
   
    #local4
    local_4 = local_layer(reshape, image_size, num_hidden, "local_4")
    drop = tf.nn.dropout(local_4, keep_prob)

    # local5
    local_5 = local_layer(drop, num_hidden, num_hidden, "local_5")
    drop = tf.nn.dropout(local_5, keep_prob)
   
    # Linear
    with tf.name_scope('softmax'):
        weights = _variable_on_cpu('weights', [num_hidden, num_classes],
                                  initializer=tf.truncated_normal_initializer(stddev=2.0 / float(num_hidden)))

        bias = _variable_on_cpu('biases', [num_classes],
                               initializer=tf.constant_initializer(0.0))

        logits = tf.add(tf.matmul(drop, weights), bias)
        
    return logits


def loss(logits, labels):
    """"
    Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int64 - [batch_size].
    Returns:
    loss: Loss tensor of type float.
    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss


def training(loss, learning_rate):
    """
    Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
    Returns:
    train_op: The Op for training.
    """
    # Add a scalar summary for the snapshot loss. (Tensor board)
    tf.scalar_summary('/loss', loss)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # decay on our learning rate as we progress with training
    learn_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=True)
    tf.scalar_summary('/learn_rate', learning_rate)
    
    # Create the gradient Adam optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))


## Train

In [36]:
def run_training():
    tf.reset_default_graph()
    with tf.Graph().as_default():
        keep_prob = tf.placeholder(tf.float32)
        
        images, labels = inputs(True, batch_size, epochs)
        
        logits = inference(images, keep_prob)
                
        loss_op = loss(logits, labels)
        
        train_op = training(loss_op, learning_rate)
        
        merge_op = tf.merge_all_summaries()
        
        eval_op = evaluation(logits, labels)

        init_op = tf.initialize_all_variables()


        sess = tf.Session()
        sess.as_default()
        
        sess.run(init_op)
        
        writer = tf.train.SummaryWriter('logs/statefarmchallenge', sess.graph)

        # Start populating the filename queue.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        

    try:
        step = 0
        
        # Add image summary for the image viewer
        tf.image_summary('/images', images)
        
        while not coord.should_stop():

            start_time = time.time()
            _, loss = sess.run([train_op, loss_op], feed_dict={keep_prob : 0.5})
            duration = time.time() - start_time 

            # Print some performance info (only for debug)
            if step % 10 == 0:
                examples_per_second = batch_size / duration
                sec_per_batch = float(duration)
                
                format_str = ('step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
                print (format_str % (step, loss_value, examples_per_sec, sec_per_batch))
                
            # Write summeries, this is the important visuals for model debugging
            if step % 100 == 0:
                summary = sess.run(merge_op)
                # implment this for model reloading. (will do later)
                # global_step = tf.get_variable(name='global_step', trainable=False)
                tf.train.SummaryWriter.add_summary(summary, step)
                
            # Save & validate the model perodically
            # if step % 1000 == 0:
                # TODO: Validation
                
            step += 1
    
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (epochs, step))
    
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        
    # Wait for threads to finish.
    coord.join(threads)
    sess.close()
        
        
run_training()

Tensor("Shape:0", shape=(3,), dtype=int32)


ValueError: Unknown attribute: '\cpu0' in '\cpu0:'

## Test 

TODO: 
- Convert test images to TF records
- Use pipeline to feed images for inference

In [ ]:
def load_test_images(path):
    """Load image to classify"""
    test_images = []
    paths = os.path.join(path, '*.jpg')
    image_files = glob.glob(paths)
    for i in paths:
        image = Image.open(i)
        test_images.append(image.getdata())
    
